Алексеев Д.П. (DSU-4, RSML-5). Домашнее задание по теме «Рекомендации на основе содержания». (№1) - СКОРРЕКТИРОВАННОЕ

1) Использовать dataset MovieLens

2) Построить рекомендации (регрессия, предсказываем оценку) на фичах:
- TF-IDF на тегах и жанрах
- Средние оценки (+ median, variance, etc.) пользователя и фильма

3) Оценить RMSE на тестовой выборке

In [1]:
# загрузим "Small" dataset: 100,000 ratings
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2022-08-23 18:39:39--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.98MB/s    in 0.2s    

2022-08-23 18:39:39 (3.98 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [2]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [3]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook

In [50]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

In [51]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [52]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [53]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [54]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [55]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [56]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


## Добавим к данным о рейтингах информацию о тегах. Учитывая, что теги проставлены только по 3683 фильмам (из 100836 записей о рейтингах), в датафрейме будет много отсутствующих значений (NaN) в поле 'tag'.

In [57]:
ratings_with_tags = ratings.merge(tags, on=['userId', 'movieId'], how='left')
ratings_with_tags.head()

,userId,movieId,rating,timestamp_x,tag,timestamp_y
0,1,1,4.0,964982703,NaN,NaN
1,1,3,4.0,964981247,NaN,NaN
2,1,6,4.0,964982224,NaN,NaN
3,1,47,5.0,964983815,NaN,NaN
4,1,50,5.0,964982931,NaN,NaN


In [58]:
ratings_with_tags['tag'].value_counts()

In Netflix queue    55
atmospheric         36
Disney              23
superhero           23
funny               23
                    ..
1960s                1
Jim Morrison         1
Navy                 1
aging                1
Heroic Bloodshed     1
Name: tag, Length: 1543, dtype: int64

## Удалим записи рейтингов с отсутствующими значениями тегов, а также колонки с отметками времени (timestamp).

In [59]:
ratings_with_tags.dropna(inplace=True)
ratings_with_tags.drop(['timestamp_x', 'timestamp_y'], axis=1, inplace=True)

In [62]:
ratings_with_tags.reset_index()
ratings_with_tags.head()

,userId,movieId,rating,tag
241,2,60756,5.0,funny
242,2,60756,5.0,Highly quotable
243,2,60756,5.0,will ferrell
252,2,89774,5.0,Boxing story
253,2,89774,5.0,MMA


In [63]:
ratings_with_tags.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3476 entries, 241 to 102673
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   3476 non-null   int64  
 1   movieId  3476 non-null   int64  
 2   rating   3476 non-null   float64
 3   tag      3476 non-null   object 
dtypes: float64(1), int64(2), object(1)
memory usage: 135.8+ KB


## Теперь преобразуем данные о жанрах фильмов в строки

In [64]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [65]:
movies['genres'] = [change_string(g) for g in movies.genres.values]

In [67]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [68]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


## Посчитаем значения оценок в разрезе пользователей (userId): средние(mean), медианные(median), дисперсию(variance)

In [69]:
# Предварительно удалим ненужные столбцы ('movieId', 'timestamp') из таблицы оценок
ratings_by_user = ratings.drop(['movieId', 'timestamp'], axis=1)

In [70]:
# сгруппируем по 'userId' и посчитаем статистики по каждому пользователю
ratings_by_user = ratings_by_user.groupby(['userId'], as_index=False).agg([np.mean, np.median, np.var])

In [71]:
ratings_by_user.reset_index()
ratings_by_user.columns = ['rating_by_user_mean', 'rating_by_user_median', 'rating_by_user_var']
ratings_by_user.head()

,rating_by_user_mean,rating_by_user_median,rating_by_user_var
userId,,,
1,4.366379,5.0,0.640077
2,3.948276,4.0,0.649015
3,2.435897,0.5,4.370783
4,3.555556,4.0,1.727132
5,3.636364,4.0,0.980973


## Теперь посчитаем то же самое в разрезе фильмов (MovieId): средние(mean), медианные(median), дисперсию(variance)

In [72]:
# Предварительно удалим ненужные колонки ('userId', 'timestamp') из таблицы оценок
ratings_by_movie = ratings.drop(['userId', 'timestamp'], axis=1)

In [73]:
# сгруппируем по 'movieId' и посчитаем статистики по каждому фильму
ratings_by_movie = ratings_by_movie.groupby(['movieId'], as_index=False).agg([np.mean, np.median, np.var])

In [74]:
ratings_by_movie.reset_index()
ratings_by_movie.columns = ['rating_by_movie_mean', 'rating_by_movie_median', 'rating_by_movie_var']
ratings_by_movie.head()

,rating_by_movie_mean,rating_by_movie_median,rating_by_movie_var
movieId,,,
1,3.920930,4.0,0.696990
2,3.431818,3.5,0.777419
3,3.259615,3.0,1.112651
4,2.357143,3.0,0.726190
5,3.071429,3.0,0.822917


## Объединим все теги по фильмам

In [75]:
def change_string_lower(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_strings = []
movies_list = []

for movie, group in ratings_with_tags.groupby('movieId'):
    tag_strings.append(' '.join([change_string_lower(s) for s in group.tag.values]))
    movies_list.append(movie)

In [76]:
movies_list[:3]

[1, 2, 3]

In [77]:
tag_strings[:3]

['pixar pixar fun', 'fantasy magicboardgame robinwilliams game', 'moldy old']

In [78]:
# Положим объединенные теги в датафрейм с ID фильмов
tags_united = pd.DataFrame({'movieId': movies_list, 'tags': tag_strings})
tags_united.reset_index()
tags_united.head(3)

,movieId,tags
0,1,pixar pixar fun
1,2,fantasy magicboardgame robinwilliams game
2,3,moldy old


## Объединим таблицу фильмов с таблицей сводных тегов по фильму (tags_united)

In [80]:
movies_with_united_tags = movies.join(tags_united.set_index('movieId'), on='movieId')

In [81]:
movies_with_united_tags.head(10)

,movieId,title,genres,tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy magicboardgame robinwilliams game
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old
3,4,Waiting to Exhale (1995),Comedy Drama Romance,NaN
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake
5,6,Heat (1995),Action Crime Thriller,NaN
6,7,Sabrina (1995),Comedy Romance,remake
7,8,Tom and Huck (1995),Adventure Children,NaN
8,9,Sudden Death (1995),Action,NaN
9,10,GoldenEye (1995),Action Adventure Thriller,NaN


## Теперь добавим статистики по фильмам (средняя, медиана, дисперсия) в датафрейм с фильмами.

In [89]:
full_movie_data = movies_with_united_tags.join(ratings_by_movie, on='movieId')

In [90]:
full_movie_data.head()

,movieId,title,genres,tags,rating_by_movie_mean,rating_by_movie_median,rating_by_movie_var
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,3.920930,4.0,0.696990
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy magicboardgame robinwilliams game,3.431818,3.5,0.777419
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old,3.259615,3.0,1.112651
3,4,Waiting to Exhale (1995),Comedy Drama Romance,NaN,2.357143,3.0,0.726190
4,5,Father of the Bride Part II (1995),Comedy,pregnancy remake,3.071429,3.0,0.822917


In [91]:
full_movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   movieId                 9742 non-null   int64  
 1   title                   9742 non-null   object 
 2   genres                  9742 non-null   object 
 3   tags                    1464 non-null   object 
 4   rating_by_movie_mean    9724 non-null   float64
 5   rating_by_movie_median  9724 non-null   float64
 6   rating_by_movie_var     6278 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 532.9+ KB


## Теперь добавим статистики по пользователям (средняя, медиана, дисперсия) из таблицы 'ratings_by_user' в таблицу с рейтингами и тегами (ratings_with_tags). [Поле 'rating' - это значение оценки конкретного пользователя конкретному фильму]

In [87]:
full_ratings = ratings_with_tags.join(ratings_by_user, on=['userId'])
full_ratings.head()

,userId,movieId,rating,tag,rating_by_user_mean,rating_by_user_median,rating_by_user_var
241,2,60756,5.0,funny,3.948276,4.0,0.649015
242,2,60756,5.0,Highly quotable,3.948276,4.0,0.649015
243,2,60756,5.0,will ferrell,3.948276,4.0,0.649015
252,2,89774,5.0,Boxing story,3.948276,4.0,0.649015
253,2,89774,5.0,MMA,3.948276,4.0,0.649015


In [88]:
full_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3476 entries, 241 to 102673
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   userId                 3476 non-null   int64  
 1   movieId                3476 non-null   int64  
 2   rating                 3476 non-null   float64
 3   tag                    3476 non-null   object 
 4   rating_by_user_mean    3476 non-null   float64
 5   rating_by_user_median  3476 non-null   float64
 6   rating_by_user_var     3476 non-null   float64
dtypes: float64(4), int64(2), object(1)
memory usage: 217.2+ KB


## Наконец, добавим к таблице 'full_ratings' посчитанные статистики по фильмам из таблицы 'full_movie_data', а также добавим объединенные теги и очищенные жанры.

In [122]:
all_data = full_ratings.join(full_movie_data.set_index('movieId'), on = 'movieId')

In [123]:
all_data.head(10)

,userId,movieId,rating,tag,rating_by_user_mean,rating_by_user_median,rating_by_user_var,title,genres,tags,rating_by_movie_mean,rating_by_movie_median,rating_by_movie_var
241,2,60756,5.0,funny,3.948276,4.0,0.649015,Step Brothers (2008),Comedy,funny highlyquotable willferrell comedy funny ...,3.553571,3.5,0.821098
242,2,60756,5.0,Highly quotable,3.948276,4.0,0.649015,Step Brothers (2008),Comedy,funny highlyquotable willferrell comedy funny ...,3.553571,3.5,0.821098
243,2,60756,5.0,will ferrell,3.948276,4.0,0.649015,Step Brothers (2008),Comedy,funny highlyquotable willferrell comedy funny ...,3.553571,3.5,0.821098
252,2,89774,5.0,Boxing story,3.948276,4.0,0.649015,Warrior (2011),Drama,boxingstory mma tomhardy,3.727273,4.0,0.918182
253,2,89774,5.0,MMA,3.948276,4.0,0.649015,Warrior (2011),Drama,boxingstory mma tomhardy,3.727273,4.0,0.918182
254,2,89774,5.0,Tom Hardy,3.948276,4.0,0.649015,Warrior (2011),Drama,boxingstory mma tomhardy,3.727273,4.0,0.918182
258,2,106782,5.0,drugs,3.948276,4.0,0.649015,"Wolf of Wall Street, The (2013)",Comedy Crime Drama,drugs leonardodicaprio martinscorsese stockmar...,3.916667,4.0,1.158019
259,2,106782,5.0,Leonardo DiCaprio,3.948276,4.0,0.649015,"Wolf of Wall Street, The (2013)",Comedy Crime Drama,drugs leonardodicaprio martinscorsese stockmar...,3.916667,4.0,1.158019
260,2,106782,5.0,Martin Scorsese,3.948276,4.0,0.649015,"Wolf of Wall Street, The (2013)",Comedy Crime Drama,drugs leonardodicaprio martinscorsese stockmar...,3.916667,4.0,1.158019
1025,7,48516,1.0,way too long,3.230263,3.5,1.767820,"Departed, The (2006)",Crime Drama Thriller,waytoolong leonardodicaprio suspense twistendi...,4.252336,4.5,0.478178


In [124]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3476 entries, 241 to 102673
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   userId                  3476 non-null   int64  
 1   movieId                 3476 non-null   int64  
 2   rating                  3476 non-null   float64
 3   tag                     3476 non-null   object 
 4   rating_by_user_mean     3476 non-null   float64
 5   rating_by_user_median   3476 non-null   float64
 6   rating_by_user_var      3476 non-null   float64
 7   title                   3476 non-null   object 
 8   genres                  3476 non-null   object 
 9   tags                    3476 non-null   object 
 10  rating_by_movie_mean    3476 non-null   float64
 11  rating_by_movie_median  3476 non-null   float64
 12  rating_by_movie_var     3279 non-null   float64
dtypes: float64(7), int64(2), object(4)
memory usage: 380.2+ KB


## Вообще, конечно, странно, что один и тот же пользователь мог поставить(разметить) одному и тому же фильму разные теги в разное время)) При этом оценки на первый взгляд одинаковые (хотя в принципе могут и отличаться, поэтому группировать по пользователю таблицу не будем, иначе это может сильно исказить результат предсказаний, т.к. конкретная оценка пользователя в поле 'rating'- это целевая переменная модели).

## Зато мы можем смело дропнуть колонку с "одиночным" тегом, ведь у нас есть теперь объединенные теги по фильму. А также нам в принципе не нужны для обучения данные с ID пользователя и названием фильма.

## Также удалим пропуски (они есть в колонке с дисперсией по фильму: rating_by_movie_var)

In [125]:
all_data.drop(['tag', 'userId', 'title'], axis=1, inplace = True)

In [126]:
all_data.dropna(inplace=True)

In [127]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3279 entries, 241 to 102673
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   movieId                 3279 non-null   int64  
 1   rating                  3279 non-null   float64
 2   rating_by_user_mean     3279 non-null   float64
 3   rating_by_user_median   3279 non-null   float64
 4   rating_by_user_var      3279 non-null   float64
 5   genres                  3279 non-null   object 
 6   tags                    3279 non-null   object 
 7   rating_by_movie_mean    3279 non-null   float64
 8   rating_by_movie_median  3279 non-null   float64
 9   rating_by_movie_var     3279 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 281.8+ KB


## Теперь всё готово для создания модели. Предварительно векторизуем текстовые признаки: жанры и объединенные теги.

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [129]:
# создадим и обучим векторизатор 
vectorizer = TfidfVectorizer()
genres = vectorizer.fit_transform(all_data['genres'])

In [130]:
genres

<3279x19 sparse matrix of type '<class 'numpy.float64'>'
	with 9054 stored elements in Compressed Sparse Row format>

In [131]:
tags = vectorizer.fit_transform(all_data['tags'])
tags

<3279x1377 sparse matrix of type '<class 'numpy.float64'>'
	with 52089 stored elements in Compressed Sparse Row format>

In [132]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [133]:
# целевая переменная - это оценка конкретного пользователя конкретному фильму, выданная в конкретный момент времени
y = all_data['rating']

In [134]:
# Попробуем сперва обучить модель с использованием только вектора тегов в качестве входящего набора признаков
# Разделим выборку на трейн и тест 
X_train, X_test, y_train, y_test = train_test_split(tags, y, test_size=0.3, random_state=42)

In [135]:
# создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [136]:
# рассчитаем прогноз на тестовой выборке и выведем RMSE
pred = model.predict(X_test)
print(mean_squared_error(pred, y_test, squared = False))

0.6130179921827614


## Как видим, использование только тегов не дало хорошего качества предсказаний, т.к. RMSE достаточно большая. Попробуем теперь только вектор жанров в качестве входящего набора признаков.


In [137]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(genres, y, test_size=0.3, random_state=42)

In [138]:
# создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [139]:
# рассчитаем прогноз на тестовой выборке и выведем RMSE
pred = model.predict(X_test)
print(mean_squared_error(pred, y_test, squared = False))

0.8087852056703371


## Мы видим, что RMSE при использовании только жанров выросла (по сравнению с использованием только тегов). 

## Попробуем расширить набор входящих признаков для регрессии, используя из предобработанного датасета (помимо жанров и тегов) ещё и рассчитанные статистики по каждому фильму и пользователю. Воспользуемся операцией hstack ("Stack arrays in sequence horizontally (column wise)"), т.к. иначе не объединить разреженные векторизированные матрицы тегов и жанров с остальными полями датафрейма (вылетает с ошибкой размерности).

In [141]:
from scipy.sparse import hstack

In [142]:
X = hstack([tags, genres, all_data[['movieId', 'rating', 'rating_by_user_mean', 'rating_by_user_median', 'rating_by_user_var', 
                                    'rating_by_movie_mean', 'rating_by_movie_median', 'rating_by_movie_var']]])

In [143]:
X

<3279x1404 sparse matrix of type '<class 'numpy.float64'>'
	with 87307 stored elements in COOrdinate format>

In [144]:
# Разделим выборку на трейн и тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [145]:
# создадим и обучим модель линейной регрессии
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [146]:
# рассчитаем прогноз на тестовой выборке и выведем RMSE
pred = model.predict(X_test)
print(mean_squared_error(pred, y_test, squared = False))

0.0013668680554416069


## RMSE уменьшилась практически до нуля, что можно считать отличным качеством предсказаний оценок моделью))